# basics

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/basics.csv')
df.head()

,gender,age,batch,avg_internet_usage,usage_type
0,Female,21 to 25,2017/ Final year,4 to 6 hours per day,Online learning
1,Male,21 to 25,2017/ Final year,4 to 6 hours per day,Information gathering for academic purposes
2,Male,21 to 25,2017/ Final year,6 to 8 hours per day,Online learning
3,Female,21 to 25,2017/ Final year,4 to 6 hours per day,Online learning
4,Female,21 to 25,2017/ Final year,4 to 6 hours per day,Lecture videos


In [3]:
df.shape

(194, 5)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   gender              194 non-null    object
 1   age                 194 non-null    object
 2   batch               194 non-null    object
 3   avg_internet_usage  194 non-null    object
 4   usage_type          194 non-null    object
dtypes: object(5)
memory usage: 7.7+ KB


no null values.

In [6]:
df['gender'].value_counts()

gender
Male                 117
Female                76
Prefer not to say      1
Name: count, dtype: int64

there is a difference in absolute gender numbers. percentages more meaningful for comparison.

In [7]:
df['age'].value_counts()

age
18 to 21    129
21 to 25     65
Name: count, dtype: int64

In [8]:
df['batch'].value_counts()

batch
2020/ First year    139
2017/ Final year     55
Name: count, dtype: int64

there is a marked difference between the number of participants from each group. so percentages are more meaningful than absolute numbers. 

In [9]:
df['avg_internet_usage'].value_counts()

avg_internet_usage
4 to 6 hours per day         61
2 to 4 hours per day         58
6 to 8 hours per day         30
0 to 2 hours per day         28
more than 8 hours per day    17
Name: count, dtype: int64

In [15]:
df[['avg_internet_usage', 'batch']].value_counts().unstack()

batch,2017/ Final year,2020/ First year
avg_internet_usage,,
0 to 2 hours per day,5,23
2 to 4 hours per day,7,51
4 to 6 hours per day,19,42
6 to 8 hours per day,16,14
more than 8 hours per day,8,9
